### Authorship
@author: Alexandre Pereira Santos <br>
alexandre.santos(at)lmu.de<br>
- many scripts in this notebook com from geemap (see below)

### Tasks
- get Global Human Settlement Layer population estimates data using the Google Earth Engine functionalities
- clip it to an AOI

### Prerequisites
- earth engine api (ee) https://developers.google.com/earth-engine/apidocs
- gee map (gee), https://geemap.org/ by Qiusheng Wu
- create an account and a Google Cloud project in Google Developer, see here: https://developers.google.com/earth-engine/guides/auth

# init

In [1]:
import ee
import geemap
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# extract coordinates from the bounding box
def get_coords(gdf):
    coords = gdf.to_crs(epsg='4326').envelope
    epsg_coords = coords.crs.to_epsg()
    transform = [coords.bounds.values[0][0], coords.bounds.values[0][1], coords.bounds.values[0][2], coords.bounds.values[0][3]]
    roi = ee.Geometry.BBox(west=transform[0], south=transform[1], east=transform[2], north=transform[3])
    rec_roi = ee.Geometry.Rectangle(transform[0],transform[1],transform[2],transform[3])
    coi = roi.centroid(maxError=1)
    return epsg_coords, roi, rec_roi, coi


In [2]:
# define a function that exports the image to Google Drive
def export_image(image, filename, crs, projection, roi):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=filename,
        folder='data',
        region=roi,
        crs=crs,
        crsTransform=projection['transform'],
        maxPixels=1e13,
        fileFormat='GeoTIFF',
        formatOptions={
            'cloudOptimized': True
        }
    )
    task.start()



# 

In [3]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-alexandresantosgeographie') # Replace with your GEE project
print(ee.__version__)

0.1.389


In [8]:
# check if the authentication is working
print(ee.Image("JRC/GHSL/P2023A/GHS_POP/1975").get("smod_code").getInfo())

None


# imports

In [5]:
#input a vector and a raster file
AOI_path = Path('../../data/processed/')

AOI_file_JAK = 'JAK_LIM_BoundingBox_AOI_A.shp' #Jakarta
AOI_file_MUM = 'MUM_LIM_BoundingBox_AOI_A.shp' #Mumbai
AOI_file_MAN = 'MAN_LIM_BoundingBox_AOI_A.shp' #Manila

#read the vector files
AOI_gdf_JAK = gpd.read_file(AOI_path/AOI_file_JAK).to_crs(epsg=4326)
AOI_gdf_MUM = gpd.read_file(AOI_path/AOI_file_MUM).to_crs(epsg=4326)
AOI_gdf_MAN = gpd.read_file(AOI_path/AOI_file_MAN).to_crs(epsg=4326)

# get the coordinates
JAK_aoi_crs, JAK_roi, JAK_rec_roi, JAK_coi = get_coords(AOI_gdf_JAK)
MUM_aoi_crs, MUM_roi, MUM_rec_roi, MUM_coi = get_coords(AOI_gdf_MUM)
MAN_aoi_crs, MAN_roi, MAN_rec_roi, MAN_coi = get_coords(AOI_gdf_MAN)

#print(coords)

In [6]:
aoi_coords = AOI_gdf_JAK.to_crs(epsg='4326').envelope
#print('N',aoi_coords.bounds.values[0][3],'S',aoi_coords.bounds.values[0][1],'W',aoi_coords.bounds.values[0][0],'E',aoi_coords.bounds.values[0][2])
transform = [aoi_coords.bounds.values[0][0], aoi_coords.bounds.values[0][1], aoi_coords.bounds.values[0][2], aoi_coords.bounds.values[0][3]]
roi = ee.Geometry.BBox(west=transform[0], south=transform[1], east=transform[2], north=transform[3])
rec_roi = ee.Geometry.Rectangle(transform[0],transform[1],transform[2],transform[3])
coi = roi.centroid(maxError=1)

In [15]:
# defining EE collection
# data from JRC GHSL https://human-settlement.emergency.copernicus.eu/ghs_pop2023.php
# earth engine collections from the JRC:
# Degree of urbanisation 1975-2023 ee.ImageCollection("JRC/GHSL/P2023A/GHS_SMOD")
# Global Built Up 1975-2023 ee.ImageCollection("JRC/GHSL/P2023A/GHS_BUILT_S")
# code based on https://developers.google.com/earth-engine/datasets/catalog/JRC_GHSL_P2023A_GHS_POP

Map = geemap.Map()
Map.centerObject(coi, zoom=9)


#baseChange = [{featureType: 'all', stylers: [{saturation: -100}, {lightness: 45}]}]
#Map.setOptions('baseChange', {'baseChange': baseChange});
image1975 = ee.Image('JRC/GHSL/P2023A/GHS_POP/1975')
image1990 = ee.Image('JRC/GHSL/P2023A/GHS_POP/1990')
image2000 = ee.Image('JRC/GHSL/P2023A/GHS_POP/2000')
image2020 = ee.Image('JRC/GHSL/P2023A/GHS_POP/2020')
populationCountVis = {
  'min': 0.0,
  'max': 100.0,
  'palette':['000004', '320A5A', '781B6C', 'BB3654', 'EC6824', 'FBB41A', 'FCFFA4']
}
#Map.setCenter(8, 48, 7);
image1975 = image1975.updateMask(image1975.gt(0))
image1990 = image1990.updateMask(image1990.gt(0))
image2000 = image2000.updateMask(image2000.gt(0))
image2020 = image2020.updateMask(image2020.gt(0))
Map.addLayer(image1975, populationCountVis, 'Population count, 1975')
Map.addLayer(image1990, populationCountVis, 'Population count, 1990')
Map.addLayer(image2020, populationCountVis, 'Population count, 2020')
Map.addLayer(roi, {}, 'ROI')
Map

Map(center=[-6.356096336666425, 106.74822478594142], controls=(WidgetControl(options=['position', 'transparent…

In [13]:
Map1975 = geemap.Map()
Map1975.centerObject(coi, zoom=9)

Map1975.addLayer(image1975, populationCountVis, 'Population count, 1975')
Map1975

Map(center=[-6.356096336666425, 106.74822478594142], controls=(WidgetControl(options=['position', 'transparent…

In [16]:
Map2000 = geemap.Map()
Map2000.centerObject(coi, zoom=9)

Map2000.addLayer(image2000, populationCountVis, 'Population count, 2000')
Map2000

Map(center=[-6.356096336666425, 106.74822478594142], controls=(WidgetControl(options=['position', 'transparent…

In [17]:
Map2020 = geemap.Map()
Map2020.centerObject(coi, zoom=9)

Map2020.addLayer(image2020, populationCountVis, 'Population count, 2000')
Map2020

Map(center=[-6.356096336666425, 106.74822478594142], controls=(WidgetControl(options=['position', 'transparent…

In [18]:
# create individual images for each of the AOIs
pop = image2020 #.select('population_count')
# JAK
JAK_pop = pop.clip(JAK_rec_roi)
JAK_crs='EPSG:'+str(JAK_aoi_crs)
# MUM
MUM_pop = pop.clip(MUM_rec_roi)
MUM_crs='EPSG:'+str(MUM_aoi_crs)
# MAN
MAN_pop = pop.clip(MAN_rec_roi)
MAN_crs='EPSG:'+str(MAN_aoi_crs)

#prepare other vars for export
projection = pop.projection().getInfo()

In [19]:
Map = geemap.Map()
Map.centerObject(JAK_coi, zoom=6)

Map.addLayer(JAK_pop, {'min': 0, 'max': 100, 'palette': ['000004', '320A5A', '781B6C', 'BB3654', 'EC6824', 'FBB41A', 'FCFFA4']}, 'population_count', True)
#Map.addLayer(MUM_pop, {'min': 0, 'max': 1, 'palette': ['0000ff', '003cd5']}, 'population_count', True)
#Map.addLayer(MAN_pop, {'min': 0, 'max': 1, 'palette': ['0000ff', '003cd5']}, 'population_count', True)
#Map.addLayer(roi, {}, 'ROI')
Map

Map(center=[-6.356096336666425, 106.74822478594142], controls=(WidgetControl(options=['position', 'transparent…

In [20]:
# export the images for the 3 cities

export_image(JAK_pop, 'JAK_POP_population_2020_GHSL', JAK_crs, projection, JAK_rec_roi)
export_image(MUM_pop, 'MUM_POP_population_2020_GHSL', MUM_crs, projection, MUM_rec_roi)
export_image(MAN_pop, 'MAN_POP_population_2020_GHSL', MAN_crs, projection, MAN_rec_roi)